In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, peak_widths
import sys
from timeit import default_timer as timer
np.set_printoptions(threshold=sys.maxsize)

from optim import Model, training_loop
from generateData import generateData, generateDataFast
from helper import computeVisibility, computeVisibility2D, pulse, matchFilt
from shape import carving

In [2]:
#####################################################################
############################# PARAMETERS ############################
#####################################################################

# laser parameters
numSpots = 150
# las_x = np.linspace(0.5, 1.5, numSpots)
# las_y = 2-las_x # illumination wall 
las_x = 1.25 * np.ones(numSpots)
las_y = np.linspace(0.25, 1.75, numSpots)
usePulse = True
pulseWidth = 1E-9 # only applicable for rectangular pulse

# detector parameters
numPixels = 150
# det_x = np.linspace(-1.5, -0.5, numPixels)
# det_y = 2+det_x # detector wall
det_x = -1.25 * np.ones(numPixels)
det_y = np.linspace(0.25, 1.75, numPixels)
detectorRes = 100E-12 # if this increases, need to subsample pulse

# noise parameters
pctNoise = 0.5
jitter = 0.5E-12

# optimization parameters
numIters = 3000
thresh = 1E-100
lr_rate = 0.001

# plotting parameters
plotData = False
plotError = False
plotSetup = True

# object parameters
pixelSize = 0.01

# constant parameters
c = 3E8
    
# miscellaneous parameters
maxDist = 3 * (np.max(det_x**2) + np.max(det_y**2))
numBins = np.ceil(((maxDist+5)/c) / detectorRes).astype(int)
t = np.linspace(0, detectorRes * numBins, numBins)

In [3]:
#####################################################################
################# GENERATE MULTIPLEXED MEASUREMENTS #################
#####################################################################
start = timer()
# compute visibility matrix
# vis_gt = np.where(np.random.rand(numPixels, numSpots) > 0.5, 1, 0)
vis_gt = computeVisibility2D(las_x, las_y, det_x, det_y, -0.25, 0.25, 0.75, 1.25, pixelSize)
# print(np.sum(vis_gt))

# compute multiplexed measurements
observations, hists = generateData(las_x, las_y, det_x, det_y, vis_gt, numBins, 
                                   plotData, plotSetup, usePulse, pctNoise, jitter, pulseWidth, t)

# solve inverse problem (reconstruct visibility matrix)
m = Model(numPixels, numSpots)
opt = torch.optim.Adam(m.parameters(), lr=lr_rate)
losses = training_loop(m, hists, observations, opt, thresh, numIters)
v_reconst = torch.where(m.vis > 0.5, 1, 0).numpy()

# plot optimization loss
if plotError:
    plt.figure(figsize=(14, 7))
    plt.plot(losses)

# compare ground truth visibility matrix
error = np.sum(np.abs(v_reconst-vis_gt))
pct_error = 100 * error / (numSpots * numPixels)
print(str(error) + ' / ' + str(numPixels*numSpots) + ' entries in visibility matrix misclassified')
print(str(pct_error) + '% of entries in visibility matrix misclassified')

onesToZero = np.sum(np.abs(v_reconst-vis_gt) * vis_gt)
print(str(onesToZero) + ' ones flipped to zeros')
print(str(100 * onesToZero / error) + '% of misclassified entries were ones flipped to zeros')
end = timer()
print(end - start)

TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
print(vis_gt)

In [ ]:
# plot for all entries
a = np.sum(vis_gt, 1)
b = np.sum(v_reconst, 1)
min_err = min(np.min(a), np.min(b))
max_err = max(np.max(a), np.max(b))
x = np.linspace(min_err, max_err); y = x
plt.figure()
plt.plot(a, b, 'o')
plt.plot(x, y, 'r')
plt.legend(['experimental', 'ideal'])
plt.xlabel('# of "on" sources (GT)')
plt.ylabel('# of "on" sources (estimated)')
plt.title('Comparison of # of "on" sources')

In [ ]:
print(np.sum(np.abs(vis_gt-v_reconst), 0))

In [ ]:
#####################################################################
#################### COMPUTE SHAPE USING CARVING ####################
#####################################################################

# carving parameters
pixelSize = 0.01 # in meters
xMin = -1.25; xMax = 1.25
eta = 0.01; zeta = 0.85; p_e = 0.5; p_o = 0.5

# compute and plot heatmap
xlocs, ylocs, heatmap, naiveMap, occupied_prob, unoccupied_prob = carving(v_reconst, las_x, las_y, det_x, det_y, 
                                                                          pixelSize, xMin, xMax, eta, zeta, p_e, p_o)
plt.figure(figsize=(6,6))
plt.title('Probabilistic Space Carving')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.contourf(xlocs, ylocs, heatmap)
plt.plot(-0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(np.linspace(-0.25, 0.25, 100), 0.75*np.ones(100))
plt.plot(np.linspace(-0.25, 0.25, 100), 1.25*np.ones(100))
plt.legend(['Ground Truth'])
plt.colorbar()

plt.figure(figsize=(6,6))
plt.title('Naive Space Carving')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.contourf(xlocs, ylocs, naiveMap)
plt.plot(-0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(np.linspace(-0.25, 0.25, 100), 0.75*np.ones(100))
plt.plot(np.linspace(-0.25, 0.25, 100), 1.25*np.ones(100))
plt.legend(['Ground Truth'])
plt.colorbar()

plt.figure(figsize=(6,6))
plt.title('Unoccupied Probability')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.contourf(xlocs, ylocs, unoccupied_prob)
plt.plot(-0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(np.linspace(-0.25, 0.25, 100), 0.75*np.ones(100))
plt.plot(np.linspace(-0.25, 0.25, 100), 1.25*np.ones(100))
plt.legend(['Ground Truth'])
plt.colorbar()

plt.figure(figsize=(6,6))
plt.title('Occupied Probability')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.contourf(xlocs, ylocs, occupied_prob)
plt.plot(-0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(0.25*np.ones(100), np.linspace(0.75, 1.25, 100))
plt.plot(np.linspace(-0.25, 0.25, 100), 0.75*np.ones(100))
plt.plot(np.linspace(-0.25, 0.25, 100), 1.25*np.ones(100))
plt.legend(['Ground Truth'])
plt.colorbar()

In [ ]:
# calculate pulse width
t, a = pulse(0, 64)
plt.plot(t, a)
peaks, _ = find_peaks(a)
results_full = peak_widths(a, peaks, rel_height=0.5)
real_width = results_full[0][0] * 100E-12
left_idx, right_idx = results_full[2:]
left_idx2 = int(np.floor(left_idx[0])); right_idx2 = int(np.ceil(right_idx[0]))
plt.plot(t[left_idx2], a[left_idx2], 'o')
plt.plot(t[right_idx2], a[right_idx2], 'o')
print('pulse width: ' + str(real_width * 1E9) + ' ns')

In [ ]:
# debug
t, a = pulse(0, 64)
# t, b = pulse(7E-10, 64)
t, b = pulse(7E-10, 64)
c = a + b

plt.plot(t, c)
sig1 = c; sig2 = a

print(matchFilt(sig1, sig2))

In [ ]:
# # naive backprojection and carving code
# def computeShape(visibility, las_x, las_y, det_x, det_y, pixelSize, minX, maxX, method):
#     numSpots = len(las_x)
#     numDets = len(det_x)
    
#     minY = np.min(np.hstack((det_y, las_y)))
#     maxY = np.max(np.hstack((det_y, las_y)))
    
#     numPixelsY = int(1 + (maxY - minY) / pixelSize)
#     numPixelsX = int(1 + (maxX - minX) / pixelSize)
    
#     ylocs = np.linspace(minY, maxY, numPixelsY)
#     xlocs = np.linspace(minX, maxX, numPixelsX)
#     heatmap = np.ones((numPixelsY, numPixelsX))
#     for i in range(numDets):
#         for j in range(numSpots):
#             if method == 'carving' and visibility[i, j] == 1:
#                 x_idx = 0
#                 for x in xlocs:
#                     x1 = las_x[j]; y1 = las_y[j]; x2 = det_x[i]; y2 = det_y[i]
#                     m = (y1 - y2) / (x1 - x2)
#                     y = m * (x - x1) + y1
#                     k = int(np.ceil((y - minY) / pixelSize)-1)
#                     heatmap[k, x_idx] = 0
#                     x_idx += 1
#             if method == 'backprojection' and visibility[i, j] == 0:
#                 x_idx = 0
#                 for x in xlocs:
#                     x1 = las_x[j]; y1 = las_y[j]; x2 = det_x[i]; y2 = det_y[i]
#                     m = (y1 - y2) / (x1 - x2)
#                     y = m * (x - x1) + y1
#                     k = int(np.ceil((y - minY) / pixelSize)-1)
#                     heatmap[k, x_idx] += 1
#                     x_idx += 1
#     return xlocs, ylocs, heatmap